In [1]:

import os
os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')


from stk_search import Searched_space
import importlib
importlib.reload(Searched_space)
import pickle
import pymongo
import pandas as pd
import numpy as np
from stk_search import Database_utils


In [2]:
#%% 
# Load the searched space
df_path = 'data/output/Full_datatset/df_total_new2023_08_20.csv'
df_precursors_path = 'Data/output/Prescursor_data/calculation_data_precursor_310823_clean.pkl'
df_total, df_precursors = Database_utils.load_data_from_file(df_path, df_precursors_path)
#df_total = Database_utils.load_data_database()

C:\Users\ma11115\OneDrive - Imperial College London\github_folder\STK_search\src\stk_search\Database_utils.py:106: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(df_path)


In [3]:
#%%
SP = Searched_space.Searched_Space(number_of_fragments=6,df=df_precursors,features_frag=df_precursors.columns[0:1],generation_type='conditional')
print(SP.features_frag)
SP.generate_interactive_condition_V2(df_total=df_total)

Index(['InChIKey'], dtype='object')


shape of the dataframe (274625, 6)
1


shape of the dataframe (274625, 6)
2


In [6]:
# load the saved dataframe with all the tested conditions
# and output it in order to select a few to add to the bactk experiment

df_SP = pd.read_pickle('data/search_space_properties_2023_09_22_09_04_06.pkl')
import pickle
from datetime import datetime
import os
def save_SP_for_EXP(df:pd.DataFrame,index_list:list,exp_name:str):
    time_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    save_dir = f'data/input/EXP_search_sapce/{exp_name}_{time_now}/'
    os.makedirs(save_dir,exist_ok=True)
    SP = Searched_space.Searched_Space(number_of_fragments=6,df=df_precursors,features_frag=df_precursors.columns[0:1],generation_type='conditional')
    SP.features_frag=SP.features_frag.append(df_precursors.columns[17:23])
    SP.features_frag=SP.features_frag.append(df_precursors.columns[1:7])

    for id in index_list:
        SP.conditions_list = df['conditions'].values[id]
        SP.syntax = list(df['syntax'].values[id])
        SP.redefine_search_space()
        print(SP.space_size)
        print(SP.syntax)
        print(SP.conditions_list)
        pickle.dump(SP,file=open(save_dir+f'/search_space_{id}.pkl','wb'))
    return SP


df_SP


,number of elements,syntax,conditions,Elements in top 5%,number of elements evaluated
0,922417564483584,"[0, 1, 2, 3, 4, 5]","[['IP (eV)'#<=#6.5], ['IP (eV)'#<=#6.5], ['IP ...",0,0
1,274625,"(0, 1, 2, 2, 1, 0)","[['IP (eV)'#<=#6.5], ['IP (eV)'#<=#6.5], ['IP ...",6.567796610169492,4.72e+02
2,274625,"(0, 1, 2, 2, 1, 0)","[['IP (eV)'#<=#6.5], ['IP (eV)'#<=#6.5], ['IP ...",6.567796610169492,4.72e+02


In [7]:
SP= save_SP_for_EXP(df_SP,[1],'exp_test')

shape of the dataframe (274625, 78)
274625
[0, 1, 2, 2, 1, 0]
[["'IP (eV)'#<=#6.5"], ["'IP (eV)'#<=#6.5"], ["'IP (eV)'#<=#6.5"], ["'IP (eV)'#<=#6.5"], ["'IP (eV)'#<=#6.5"], ["'IP (eV)'#<=#6.5"]]


IndexError: list index out of range

In [14]:
def plot_histogram_fragment(df,df_total,number_of_fragments):
        def plot_histogram_fragment(column_name,df,df_total,number_of_fragments):
            fig, ax = plt.subplots(3, 2, figsize=(6, 6),sharex='col', sharey='row')
            # ax share x axis and y axis
            #ax = ax.flatten()

            for i in range(number_of_fragments):
                ax[i // 2, i % 2].hist(
                    df_total[f"{column_name}_{i}"],
                    bins=20,
                    edgecolor="black",
                    density=True,
                    label = "all"
                )
                ax[i // 2, i % 2].hist(
                    df[f"{column_name}_{i}"],
                    bins=20,
                    edgecolor="black",
                    density=True,
                    label = "current"
                )
                ax[i // 2, i % 2].set_xlabel(f"{column_name}_{i}")
                ax[i // 2, i % 2].legend()
                #ax[i // 2, i % 2].set_ylim([0, 1])
                #ax[i // 2, i % 2].set_ylabel("Frequency")
                #ax[i // 3, i % 3].set_title(f"Histogram of {column_name}_{i}")
            # set xlabel and y label for the last row

            plt.tight_layout()
            plt.show()

        # Interactive widget for column selection
        columns_dropdown = widgets.Dropdown(
            options=[x.replace('_0','') for x in df.select_dtypes(include=["int", "float"]).columns if '_0' in x],
            description="Value:",
        )
        # Set up the layout of the widgets

        vbox_layout = Layout(display="flex", flex_flow="row", align_items="flex-start")
        # Display the widget
        interact(plot_histogram, column_name=columns_dropdown, layout=vbox_layout)
        plt.show()
plot_histogram_precursor(SP,df_total=df_total)

TypeError: plot_histogram_fragment() missing 1 required positional argument: 'number_of_fragments'

In [290]:
def plot_histogram_2(column_name,df_current_2):
    fig, ax = plt.subplots(3, 2, figsize=(6, 6),sharex='col', sharey='row')
    for i in range(SP.number_of_fragments):
        ax[i // 2, i % 2].hist(
            df_total[f"{column_name}_{i}"],
            bins=20,
            edgecolor="black",
            density=True,
            label = "all"
        )
        ax[i // 2, i % 2].hist(
            df_current_2[f"{column_name}_{i}"],
            bins=20,
            edgecolor="black",
            density=True,
            label = "current"
        )
        ax[i // 2, i % 2].set_xlabel(f"{column_name}_{i}")
        ax[i // 2, i % 2].legend()
    # set xlabel and y label for the last row

    plt.tight_layout()
    plt.show()
df_total=df_test
df= SP.check_df_for_element_from_SP_(df_to_check=df_total)
# Interactive widget for column selection
columns_dropdown_2 = widgets.Dropdown(
    options=[x.replace('_0','') for x in df_total.select_dtypes(include=["int", "float"]).columns if '_0' in x],
    description="Value:",
)
hist_widget = interactive(plot_histogram_2, column_name=widgets.fixed('Atom_num'),df_current_2=widgets.fixed(df))
display(hist_widget)
#wait for the user to select the syntax
input("Press Enter to continue...")
hist_widget.kwargs['df_current_2']=SP.check_df_for_element_from_SP_(df_to_check=df_total)[:5]

hist_widget.update()

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [11]:
list_col = [f'InChIKey_{x}' for x in range(6)]
list_col.append('target')
df_total[list_col].to_csv('Data/output/Full_datatset/data_for_adam.csv',index=False)